- areaを複数から一つだけにして、weatherデータと結合をしてみる
- k-fold,earlystopingの採用
- 

In [1]:
import pandas as pd
from ydata_profiling import ProfileReport
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import numpy as np
import seaborn as sns
%matplotlib inline


In [2]:
test_df = pd.read_csv("test.csv")
train_df = pd.read_csv("train.csv")
weather = pd.read_csv("weather.csv")
sample = pd.read_csv("sample_submission.csv")

In [3]:
weather

,date,mean_temp,max_temp,max_temp_time,min_temp,min_temp_time,sum_rain,sun_time,mean_humid,area
0,20041106,13.1,20.1,2004/11/6 12:50,8.1,2004/11/6 05:31,5.0,1.6,77.0,青森
1,20041107,9.6,12.5,2004/11/7 13:40,5.4,2004/11/7 22:17,0.0,4.1,63.0,青森
2,20041108,9.0,15.5,2004/11/8 12:51,3.2,2004/11/8 06:28,0.0,8.9,72.0,青森
3,20041109,12.2,17.7,2004/11/9 14:40,5.7,2004/11/9 02:49,1.5,1.2,85.0,青森
4,20041110,11.6,16.4,2004/11/10 14:42,6.9,2004/11/10 06:23,0.0,7.5,89.0,青森
...,...,...,...,...,...,...,...,...,...,...
204315,20220426,21.3,23.6,2022/4/26 14:27,17.9,2022/4/26 04:17,2.5,0.0,89.0,東京
204316,20220427,23.0,27.7,2022/4/27 13:12,18.9,2022/4/28 0:00,0.5,1.6,79.0,東京
204317,20220428,16.9,21.1,2022/4/28 15:03,14.2,2022/4/28 22:40,0.0,5.5,57.0,東京
204318,20220429,12.9,16.4,2022/4/29 09:21,9.0,2022/4/29 23:55,40.0,0.2,86.0,東京


In [4]:
train_df

,kind,date,amount,mode_price,area
0,だいこん,20051107,201445,735.0,千葉
1,だいこん,20051108,189660,840.0,千葉_各地_青森
2,だいこん,20051110,218166,735.0,千葉_各地_青森
3,だいこん,20051111,182624,682.5,千葉_青森
4,だいこん,20051112,220691,682.5,千葉_青森
...,...,...,...,...,...
97777,しめじ,20170526,17231,48.5,新潟_長野
97778,しめじ,20170527,17105,48.5,新潟_長野
97779,しめじ,20170529,12406,48.5,新潟_長野
97780,しめじ,20170530,12320,48.5,新潟_長野


In [5]:
sample

,kind,date,mode_price
0,だいこん,20220502,0
1,だいこん,20220506,0
2,だいこん,20220507,0
3,だいこん,20220509,0
4,だいこん,20220510,0
...,...,...,...
315,ミニトマト,20220526,0
316,ミニトマト,20220527,0
317,ミニトマト,20220528,0
318,ミニトマト,20220530,0


In [6]:
#profile = ProfileReport(train_df, title="Profiling Report")
#profile.to_notebook_iframe()
#profile.to_file("your_report.html")
# amount削除
train_df = train_df.drop(columns=['amount'])


In [7]:

#一つ目の産出地のみ出力する
def split_location(series_area):
    x=series_area.str.split("_")
    ret_date = x.apply(lambda x : x[0])
    return ret_date
    

In [8]:


# カテゴリ変数をcategory型に変換 
# 日付データを年数、月、日に分割
def Data_cre(df):

    # areaを一地域のみのデータにする
    df["area"]  = split_location(df["area"])

    # weatherデータと結合
    df= pd.merge(df, weather, on=['date', 'area'], how='left')

    # dateをdatetimeに変更して、年、月、日のデータを取得する
    df["date"] = df["date"].astype("str")
    df["date"] = pd.to_datetime(df["date"])
    
    df["year"] = df["date"].dt.year
    df["month"] = df["date"].dt.month
    df["day"] = df["date"].dt.day
    
    # いらないカラムを削除
    df = df.drop(columns=['date',"max_temp_time","min_temp_time"])
    
    #カテゴリー化
    df['kind'] = df['kind'].astype('category')
    df['area'] = df['area'].astype('category')
    return df

train_df = Data_cre(train_df)
test_df = Data_cre(test_df) 



In [9]:
train_df["area"]

0        千葉
1        千葉
2        千葉
3        千葉
4        千葉
         ..
97777    新潟
97778    新潟
97779    新潟
97780    新潟
97781    新潟
Name: area, Length: 97782, dtype: category
Categories (37, object): ['アメリカ', 'カナダ', 'トンガ', 'ニュージーランド', ..., '静岡', '香川', '高知', '鹿児島']

In [10]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97782 entries, 0 to 97781
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   kind        97782 non-null  category
 1   mode_price  97782 non-null  float64 
 2   area        97782 non-null  category
 3   year        97782 non-null  int32   
 4   month       97782 non-null  int32   
 5   day         97782 non-null  int32   
dtypes: category(2), float64(1), int32(3)
memory usage: 2.1 MB


In [11]:
X = train_df.drop(['mode_price'], axis=1)
y = train_df['mode_price']
# 学習データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=0)

In [12]:
### chatgpt
from sklearn.model_selection import KFold
import numpy as np
import lightgbm as lgb

models = {}  # kind → model list or averaged は辞書に保存

K = 5  # ← Fold数

for kd, group in train_df.groupby("kind", observed=False):

    print(f"\n===== {kd} モデルをKFold({K})で学習中 =====")

    X = group.drop(columns=["mode_price"],axis=1)
    y = group["mode_price"]

    kf = KFold(n_splits=K, shuffle=True, random_state=42)

    fold_models = []  # foldごとのモデルを保存

    for fold, (train_index, valid_index) in enumerate(kf.split(X)):
        print(f"Fold {fold+1}/{K}")

        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

        train_data = lgb.Dataset(X_train, y_train)
        valid_data = lgb.Dataset(X_valid, y_valid)

        params = {
            'objective': 'regression',
            'num_leaves': 5,
            'seed': fold,
            'verbose': -1,
        }

        model = lgb.train(
            params,
            train_data,
            num_boost_round=1000,
            valid_sets=[train_data, valid_data],
            valid_names=['train', 'valid'],
            callbacks=[
                lgb.early_stopping(stopping_rounds=20, verbose=True),
                lgb.log_evaluation(50)
            ],
        )
        
        fold_models.append(model)

    models[kd] = fold_models  # kindごとにfoldモデルを保存



===== いんげん モデルをKFold(5)で学習中 =====
Fold 1/5
Training until validation scores don't improve for 20 rounds
[50]	train's l2: 210723	valid's l2: 301581
[100]	train's l2: 190282	valid's l2: 287783
[150]	train's l2: 173979	valid's l2: 274636
[200]	train's l2: 164771	valid's l2: 263866
[250]	train's l2: 158794	valid's l2: 257718
Early stopping, best iteration is:
[271]	train's l2: 156647	valid's l2: 256878
Fold 2/5
Training until validation scores don't improve for 20 rounds
[50]	train's l2: 229486	valid's l2: 237225
[100]	train's l2: 208723	valid's l2: 223677
[150]	train's l2: 196373	valid's l2: 216215
Early stopping, best iteration is:
[161]	train's l2: 194006	valid's l2: 214933
Fold 3/5
Training until validation scores don't improve for 20 rounds
[50]	train's l2: 238447	valid's l2: 189917
[100]	train's l2: 213636	valid's l2: 178306
[150]	train's l2: 200054	valid's l2: 174243
Early stopping, best iteration is:
[147]	train's l2: 200783	valid's l2: 174041
Fold 4/5
Training until validation sc

In [13]:
import numpy as np

pred_list = []

for kd, group in test_df.groupby("kind"):

    fold_models = models[kd]  # ← foldモデルが複数入っている
    X_test = group.copy()

    # 全 fold モデルの予測を行い平均
    fold_pred = np.mean([m.predict(X_test) for m in fold_models], axis=0)

    X_test["mode_price"] = fold_pred
    pred_list.append(X_test)

# まとめ・並び順合わせ
result = pd.concat(pred_list).sort_index()

# 日付整形（year,month,day → YYYYMMDD）
result["date"] = (
    result["year"].astype(str)
    + result["month"].astype(str).str.zfill(2)
    + result["day"].astype(str).str.zfill(2)
)
result["date"] = result["date"].astype(int)

# 提出形式に整形
submit = result[["kind", "date", "mode_price"]]
submit.to_csv("submit.csv", index=False)

submit.head()

C:\Users\ND013\AppData\Local\Temp\ipykernel_1464\1594108802.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for kd, group in test_df.groupby("kind"):


,kind,date,mode_price
0,だいこん,20220502,1070.802017
1,だいこん,20220506,1045.871474
2,だいこん,20220507,1044.039995
3,だいこん,20220509,1025.844732
4,だいこん,20220510,1028.738315
